In [16]:
import sqlite3
from faker import Faker
from contextlib import closing
import random
from datetime import datetime, timedelta
import pandas as pd
DB_PATH = r"C:\Users\DELL\Desktop\nirupa\Guvi\MINI_PROJECT_GUVI"
class PlacementDB:
    def __init__(self, db_name: str = "placement_eligible.db"):
        self.conn = sqlite3.connect(db_name)
        self.conn.execute("PRAGMA foreign_keys = ON")
        self.create_tables()

    
    
    def create_tables(self):
        sa = """
        DROP TABLE IF EXISTS placement_table;
        DROP TABLE IF EXISTS soft_skills_table;
        DROP TABLE IF EXISTS programming_table;
        DROP TABLE IF EXISTS students;
        --Students table creation
    
        CREATE TABLE IF NOT EXISTS students (
            student_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER,
            gender TEXT,
            email TEXT UNIQUE,
            phone TEXT,
            enrollment_year INTEGER,
            course_batch TEXT,
            city TEXT,
            graduation_year INTEGER
        );
          --programming_table creation
        CREATE TABLE IF NOT EXISTS programming_table (
            programming_id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id INTEGER,
            language TEXT,
            problems_solved INTEGER,
            assessments_completed INTEGER,
            mini_projects INTEGER,
            certifications_earned INTEGER,
            latest_project_score INTEGER,
            FOREIGN KEY(student_id) REFERENCES students(student_id)
        );
            --softskills table creation
        CREATE TABLE IF NOT EXISTS soft_skills_table (
            soft_skill_id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id INTEGER,
            communication INTEGER,
            teamwork INTEGER,
            presentation INTEGER,
            leadership INTEGER,
            critical_thinking INTEGER,
            interpersonal_skills INTEGER,
            FOREIGN KEY(student_id) REFERENCES students(student_id)
        );
            --placement table creation
        CREATE TABLE IF NOT EXISTS placement_table (
            placement_id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id INTEGER,
            mock_interview_score INTEGER,
            internships_completed INTEGER,
            placement_status TEXT,
            company_name TEXT,
            placement_package REAL,
            interview_rounds_cleared INTEGER,
            placement_date TEXT,
            FOREIGN KEY(student_id) REFERENCES students(student_id)
        );
        """
        with closing(self.conn.cursor()) as cur:
            cur.executescript(sa)
            self.conn.commit()

   
    def insert_fake_students(self, n=500):
        fake = Faker()
        fake.unique.clear() 
        rows = []
        for _ in range(n):
            gender = random.choice(['Male', 'Female', 'Other'])
            name = fake.name_male() if gender == 'Male' else \
               fake.name_female() if gender == 'Female' else fake.name()
            enroll_year = random.randint(2012, 2016)
            rows.append((
                name,
                random.randint(18, 25),
                gender,
                fake.unique.email(),
                fake.phone_number(),
                enroll_year,
                f"Batch-{enroll_year}",
                fake.city(),
                enroll_year + 4))
         

        with closing(self.conn.cursor()) as cur:
            cur.executemany("""
            INSERT INTO students
            (name, age, gender, email, phone, enrollment_year,
             course_batch, city, graduation_year)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, rows)
        self.conn.commit()

    def insert_fake_programming_data(self):
        language= ['Python', 'Java', 'C++', 'SQL']
        with closing(self.conn.cursor()) as cur:
            cur.execute("SELECT student_id FROM students")
            students = cur.fetchall()

            rows = []
            for (sid,) in students:
                rows.append((
                    sid,
                    random.choice(language),
                    random.randint(10, 100),
                    random.randint(1, 10),
                    random.randint(0, 5),
                    random.randint(0, 3),
                    random.randint(0, 100)
                ))

            cur.executemany("""
                INSERT INTO programming_table
                (student_id, language, problems_solved, assessments_completed,
                 mini_projects, certifications_earned, latest_project_score)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, rows)
            self.conn.commit()

    def insert_fake_soft_skills_data(self):
        with closing(self.conn.cursor()) as cur:
            cur.execute("SELECT student_id FROM students")
            students = cur.fetchall()

            rows = []
            for (sid,) in students:
                rows.append((
                    sid,
                    *[random.randint(50, 100) for _ in range(6)]
                ))

            cur.executemany("""
                INSERT INTO soft_skills_table
                (student_id, communication, teamwork, presentation,
                 leadership, critical_thinking, interpersonal_skills)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, rows)
            self.conn.commit()

    def insert_fake_placement_data(self):
        fake      = Faker()
        statuses  = ['Ready', 'Not Ready', 'Placed']
        companies = ['Google', 'Microsoft', 'Infosys', 'TCS', 'Capgemini', None]

        with closing(self.conn.cursor()) as cur:
            cur.execute("SELECT student_id FROM students")
            students = cur.fetchall()

            rows = []
            for (sid,) in students:
                status   = random.choice(statuses)
                company  = random.choice(companies) if status == 'Placed' else None
                package  = round(random.uniform(3.0, 15.0), 2) if status == 'Placed' else None
                rounds   = random.randint(1, 5) if status == 'Placed' else 0
                date = fake.date_between('-2y', 'today').isoformat() if status == 'Placed' else None

                rows.append((
                    sid,
                    random.randint(40, 100),      
                    random.randint(0, 3),         
                    status,
                    company,
                    package,
                    rounds,
                    date
                ))

            cur.executemany("""
                INSERT INTO placement_table
                (student_id, mock_interview_score, internships_completed,
                 placement_status, company_name, placement_package,
                 interview_rounds_cleared, placement_date)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, rows)
            self.conn.commit()
       
    
    def close(self):
        self.conn.close()


if __name__ == "__main__":
    db = PlacementDB()
    db.insert_fake_students(500)
    db.insert_fake_programming_data()
    db.insert_fake_soft_skills_data()
    db.insert_fake_placement_data()
    db.close()
    print(" Database created and populated with sample data.")

✅ Database created and populated with sample data.
